# DeepDreaming with TensorFlow

>[Loading the model graph](#loading)

>[Naive feature visualization](#naive)

>[Multiscale image generation](#multiscale)

>[Laplacian Pyramid Gradient Normalization](#laplacian)

>[Playing with feature visualzations](#playing)

>[DeepDream](#deepdream)



This notebook demonstrates a number of Convolutional Neural Network image generation techniques implemented with TensorFlow for fun and science:

- visualize individual feature channels and their combinations to explore the space of patterns learned by the neural network (see [GoogLeNet](http://storage.googleapis.com/deepdream/visualz/tensorflow_inception/index.html) and [VGG16](http://storage.googleapis.com/deepdream/visualz/vgg16/index.html) galleries)
- embed TensorBoard graph visualizations into Jupyter notebooks
- produce high-resolution images with tiled computation ([example](http://storage.googleapis.com/deepdream/pilatus_flowers.jpg))
- use Laplacian Pyramid Gradient Normalization to produce smooth and colorful visuals at low cost
- generate DeepDream-like images with TensorFlow (DogSlugs included)


The network under examination is the [GoogLeNet architecture](https://arxiv.org/pdf/1409.4842.pdf), trained to classify images into one of 1000 categories of the [ImageNet](http://image-net.org/) dataset. It consists of a set of layers that apply a sequence of transformations to the input image. The parameters of these transformations were determined during the training process by a variant of gradient descent algorithm. The internal image representations may seem obscure, but it is possible to visualize and interpret them. In this notebook we are going to present a few tricks that allow to make these visualizations both efficient to generate and even beautiful. Impatient readers can start with exploring the full galleries of images generated by the method described here for [GoogLeNet](http://storage.googleapis.com/deepdream/visualz/tensorflow_inception/index.html) and [VGG16](http://storage.googleapis.com/deepdream/visualz/vgg16/index.html) architectures.

In [1]:
%tensorflow_version 1.x
# boilerplate code
from __future__ import print_function
import os
!pip -q install imageio
import imageio as io
# maybe choose between imageio and BytesIO?
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import matplotlib.pyplot as plt
from skimage.transform import resize as imresize

import tensorflow as tf

TensorFlow 1.x selected.


<a id='loading'></a>
## Loading the model graph

The pretrained network can be downloaded [here](https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip). Unpack the `tensorflow_inception_graph.pb` file from the archive and set its path to `model_fn` variable. Alternatively you can uncomment and run the following cell to download the network:

In [2]:
!wget https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip && unzip inception5h.zip

--2020-11-01 15:15:44--  https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.240, 172.217.8.16, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49937555 (48M) [application/zip]
Saving to: ‘inception5h.zip’

inception5h.zip     100%[===================>]  47.62M  82.5MB/s    in 0.6s    

2020-11-01 15:15:44 (82.5 MB/s) - ‘inception5h.zip’ saved [49937555/49937555]

Archive:  inception5h.zip
  inflating: imagenet_comp_graph_label_strings.txt  
  inflating: tensorflow_inception_graph.pb  
  inflating: LICENSE                 


In [3]:
#sess.close()
model_fn = 'tensorflow_inception_graph.pb'

# creating TensorFlow session and loading the model from the model_fn file 
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})

Instructions for updating:
Use tf.gfile.GFile.


In [4]:

# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

To take a glimpse into the kinds of patterns that the network learned to recognize, we will try to generate images that maximize the sum of activations of particular channel of a particular convolutional layer of the neural network. The network we explore contains many convolutional layers, each of which outputs tens to hundreds of feature channels, so we have plenty of patterns to explore.

In [5]:
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))


Number of layers 59
Total number of feature channels: 7548


In [6]:
def T(layer):
    '''Helper for getting layer output tensor'''
    return graph.get_tensor_by_name("import/%s:0"%layer)

In [7]:
# TO-DO : fix index calling so import is not twice printed with T(layer)

layer=layers[0]
print(layer)
layer = layer.split("/")[1]
print(layer)

import/conv2d0_pre_relu/conv
conv2d0_pre_relu


In [8]:
T(layer)

<tf.Tensor 'import/conv2d0_pre_relu:0' shape=(?, ?, ?, 64) dtype=float32>

In [9]:
for l, layer in enumerate(layers):
    layer = layer.split("/")[1]
    num_channels = T(layer).shape[3]
    print(l, layer, num_channels)

0 conv2d0_pre_relu 64
1 conv2d1_pre_relu 64
2 conv2d2_pre_relu 192
3 mixed3a_1x1_pre_relu 64
4 mixed3a_3x3_bottleneck_pre_relu 96
5 mixed3a_3x3_pre_relu 128
6 mixed3a_5x5_bottleneck_pre_relu 16
7 mixed3a_5x5_pre_relu 32
8 mixed3a_pool_reduce_pre_relu 32
9 mixed3b_1x1_pre_relu 128
10 mixed3b_3x3_bottleneck_pre_relu 128
11 mixed3b_3x3_pre_relu 192
12 mixed3b_5x5_bottleneck_pre_relu 32
13 mixed3b_5x5_pre_relu 96
14 mixed3b_pool_reduce_pre_relu 64
15 mixed4a_1x1_pre_relu 192
16 mixed4a_3x3_bottleneck_pre_relu 96
17 mixed4a_3x3_pre_relu 204
18 mixed4a_5x5_bottleneck_pre_relu 16
19 mixed4a_5x5_pre_relu 48
20 mixed4a_pool_reduce_pre_relu 64
21 mixed4b_1x1_pre_relu 160
22 mixed4b_3x3_bottleneck_pre_relu 112
23 mixed4b_3x3_pre_relu 224
24 mixed4b_5x5_bottleneck_pre_relu 24
25 mixed4b_5x5_pre_relu 64
26 mixed4b_pool_reduce_pre_relu 64
27 mixed4c_1x1_pre_relu 128
28 mixed4c_3x3_bottleneck_pre_relu 128
29 mixed4c_3x3_pre_relu 256
30 mixed4c_5x5_bottleneck_pre_relu 24
31 mixed4c_5x5_pre_relu 64
32 

<a id='naive'></a>
## Naive feature visualization

Let's start with a naive way of visualizing these. Image-space gradient ascent!

In [10]:
# Picking some internal layer. Note that we use outputs before applying the ReLU nonlinearity
# to have non-zero gradients for features with negative initial activations.
layer = 'mixed4d_3x3_bottleneck_pre_relu'
print(layers[37])
channel = 139 # picking some feature channel to visualize

# start with a gray image with a little noise
img_noise = np.random.uniform(size=(224,224,3)) + 100.0


import/mixed4d_5x5_pre_relu/conv


In [11]:

def showarray(a, fmt='jpeg'):
    '''create a jpeg file from an array a and visualize it'''
    # clip the values to be between 0 and 255
    a = np.uint8(np.clip(a, 0, 1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def visstd(a, s=0.1):
    '''Normalize the image range for visualization'''
    return (a-a.mean())/max(a.std(), 1e-4)*s + 0.5


<a id="multiscale"></a>
## Multiscale image generation

Looks like the network wants to show us something interesting! Let's help it. We are going to apply gradient ascent on multiple scales. Details formed on smaller scale will be upscaled and augmented with additional details on the next scale.

With multiscale image generation it may be tempting to set the number of octaves to some high value to produce wallpaper-sized images. Storing network activations and backprop values will quickly run out of GPU memory in this case. There is a simple trick to avoid this: split the image into smaller tiles and compute each tile gradient independently. Applying random shifts to the image before every iteration helps avoid tile seams and improves the overall image quality.

In [12]:
def tffunc(*argtypes):
    '''Helper that transforms TF-graph generating function into a regular one.
    See "resize" function below.
    '''
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

# Helper function that uses TF to resize an image
def resize(img, size):
    img = tf.expand_dims(img, 0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]
resize = tffunc(np.float32, np.int32)(resize)


def calc_grad_tiled(img, t_grad, i=None, tile_size=None):
    '''Compute the value of tensor t_grad over the image in a tiled way.
    Random shifts are applied to the image to blur tile boundaries over 
    multiple iterations.'''
    # done on logic that keeping tile size constant with image size will be more stable
    if tile_size == None:
        tile_size = len(img[1])
    sz = tile_size
    h, w = img.shape[:2] # size of the image
    # Probably causing jitter - TRY having each iteration end on same position
    # TO DO - also causing jitter is gradients (likely), so include prev image as target/weight and/or interp between frames
    np.random.seed(i)
    sx, sy = np.random.randint(sz, size=2) # random shift numbers generated
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0) #shift the whole image. np.roll = Roll array elements along a given axis
    grad = np.zeros_like(img)
    #blend gradients of prior image
    for y in range(0, max(h-sz//2, sz),sz):
        for x in range(0, max(w-sz//2, sz),sz):
            sub = img_shift[y:y+sz,x:x+sz] # get the image patch (tile)
            g = sess.run(t_grad, {t_input:sub}) # calculate the gradient only in the image patch not in the whole image!
            grad[y:y+sz,x:x+sz] = g # put the whole gradient together from the tiled gradients g
    return np.roll(np.roll(grad, -sx, 1), -sy, 0) # shift back

<a id="laplacian"></a>
## Laplacian Pyramid Gradient Normalization

This looks better, but the resulting images mostly contain high frequencies. Can we improve it? One way is to add a smoothness prior into the optimization objective. This will effectively blur the image a little every iteration, suppressing the higher frequencies, so that the lower frequencies can catch up. This will require more iterations to produce a nice image. Why don't we just boost lower frequencies of the gradient instead? One way to achieve this is through the [Laplacian pyramid](https://en.wikipedia.org/wiki/Pyramid_%28image_processing%29#Laplacian_pyramid) decomposition. We call the resulting technique _Laplacian Pyramid Gradient Normalization_.

In [13]:
k = np.float32([1,4,6,4,1])
k = np.outer(k, k)
k5x5 = k[:,:,None,None]/k.sum()*np.eye(3, dtype=np.float32)

def lap_split(img):
    '''Split the image into lo and hi frequency components'''
    with tf.name_scope('split'):
        lo = tf.nn.conv2d(img, k5x5, [1,2,2,1], 'SAME')
        lo2 = tf.nn.conv2d_transpose(lo, k5x5*4, tf.shape(img), [1,2,2,1])
        hi = img-lo2
    return lo, hi

def lap_split_n(img, n):
    '''Build Laplacian pyramid with n splits'''
    levels = []
    for i in range(n):
        img, hi = lap_split(img)
        levels.append(hi)
    levels.append(img)
    return levels[::-1]

def lap_merge(levels):
    '''Merge Laplacian pyramid'''
    img = levels[0]
    for hi in levels[1:]:
        with tf.name_scope('merge'):
            img = tf.nn.conv2d_transpose(img, k5x5*4, tf.shape(hi), [1,2,2,1]) + hi
    return img

def normalize_std(img, eps=1e-10):
    '''Normalize image by making its standard deviation = 1.0'''
    with tf.name_scope('normalize'):
        std = tf.sqrt(tf.reduce_mean(tf.square(img)))
        return img/tf.maximum(std, eps)

def lap_normalize(img, scale_n=4):
    '''Perform the Laplacian pyramid normalization.'''
    img = tf.expand_dims(img,0)
    tlevels = lap_split_n(img, scale_n)
    tlevels = list(map(normalize_std, tlevels))
    out = lap_merge(tlevels)
    return out[0,:,:,:]

# Showing the lap_normalize graph with TensorBoard
lap_graph = tf.Graph()
with lap_graph.as_default():
    lap_in = tf.placeholder(np.float32, name='lap_in')
    lap_out = lap_normalize(lap_in)
show_graph(lap_graph)

In [14]:
#image preprocessing
def preprocess(img, crop=True, changesize=True, dsize=(224, 224)):
    """Summary

    Parameters
    ----------
    img : TYPE
        Description
    crop : bool, optional
        Description
    resize : bool, optional
        Description
    dsize : tuple, optional
        Description

    Returns
    -------
    TYPE
        Description
    """
    if img.dtype == np.uint8:
        img = img / 255.0

    if crop:
        short_edge = min(img.shape[:2])
        yy = int((img.shape[0] - short_edge) / 2)
        xx = int((img.shape[1] - short_edge) / 2)
        crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    else:
        crop_img = img

    if changesize:
        norm_img = imresize(crop_img, dsize, preserve_range=True)
    else:
        norm_img = crop_img

    return (norm_img).astype(np.float32)
        
# Note: entered additional t_score for summative gradient between 2 points. 
def render_lapnorm2(t_obj, t_obj2=tf.zeros([0]), img0=img_noise, score='mean', vis='clear', visfunc=visstd,
                   iter_n=20, step=1.0, octave_n=3, octave_scale=1.4, lap_n=4, grady='double', morph_im=None, file_dir=os.getcwd()):
    if score == 'mean':
        t_score = tf.reduce_mean(t_obj) # defining the optimization objective
        t_score2 = tf.reduce_mean(t_obj2) # defining the optimization objective
    if score == 'sum':
        t_score = tf.reduce_sum(t_obj) # defining the optimization objective
        t_score2 = tf.reduce_sum(t_obj2) # defining the optimization objective
    t_grad = tf.gradients(t_score+t_score2, t_input, grad_ys=0.5)[0] # behold the power of automatic differentiation!
    # build the laplacian normalization graph
    lap_norm_func = tffunc(np.float32)(partial(lap_normalize, scale_n=lap_n))
    img = img0.copy()
    for octave in range(octave_n):
        if octave>0:
            hw = np.float32(img.shape[:2])*octave_scale
            img = resize(img, np.int32(hw))
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad, i)
            if grady == 'double':
                g_prev = (img + g)/2
                g += sess.run(t_grad, {t_input: g_prev})
            g = lap_norm_func(g)
            img += g*step
            print('.', end = ' ')
        if vis == 'clear':
            clear_output()
            showarray(visfunc(img))
        if vis == 'all':
            showarray(visfunc(img))
        if vis == 'zoomin':
            crop = 25
            img_c = img
            # just for more color
            img_c += img*0.2
            img_c = img_c[crop:-crop, crop:-crop]
            print(img_c.shape)
            img_c = imresize(img_c, (len(img[0]), len(img[1]), 3), order=4,
                 clip=False, preserve_range=True)#[np.newaxis].astype(np.float32)
            img = img_c
            print(img.shape)
    if vis == 'fin':
        showarray(visfunc(img))
    if vis == 'save':
        img = visfunc(img)
        img = np.uint8(np.clip(img, 0, 1)*255)
        #io.imwrite(file_dir+'/interp%.3f.png' % (wght1), img)
        
        #for colab
        fname = '/interp%.3f.png' % (wght1)
        img_out = PIL.Image.fromarray(img)
        img_out.save(fname)
        files.download(fname)
        
    if vis == 'zoomin':
        img_c = visfunc(img_c)
        img_c = np.uint8(np.clip(img_c,0, 1)*255)
        #io.imwrite(file_dir+'/zinterp%.3f.png' % (wght1), img_c)
        
        #for colab
        fname = '/zinterp%.3f.png' % (wght1)
        img_out = PIL.Image.fromarray(img)
        img_out.save(fname)
        files.download(fname)
    if vis == 'morph':
        morph_im = imresize(morph_im, (img.shape), preserve_range=True)
        img = visfunc(img)
        img = np.uint8(np.clip(img, 0, 1)*255)
        imgs = np.asarray(morph(morph_im, img)).astype(np.uint8)
        morph_im = img
        for i, _ in enumerate(imgs):
            #io.imwrite(file_dir+'/interp%.3f.png' % (wght1), img)

            #for colab
            fname = '/minterp%.3f%.2d.png' % (wght1, i)
            img_out = PIL.Image.fromarray(imgs[i])
            img_out.save(fname)
            files.download(fname)
        return morph_im


In [15]:
def morph(z1, z2, n_frame = 10):
    '''
    this saves an animated gif from two latent states z1 and z2
    n_frame: number of states in between z1 and z2 morphing effect, exclusive of z1 and z2
    duration1, duration2, control how long z1 and z2 are shown.  duration controls frame speed, in seconds
    '''
    delta_z = (z2-z1) / (n_frame+1)
    total_frames = n_frame + 2
    images = []
    for i in range(total_frames):
        z = z1 + delta_z*float(i)
        images.append(z)
        print("processing image ", i)
    return images
    

In [16]:
render_lapnorm2(T(layer)[:,:,:,channel], vis='all', grady='double')

KeyboardInterrupt: ignored

<a id="playing"></a>
## Playing with feature visualizations

We got a nice smooth image using only 10 iterations per octave. In case of running on GPU this takes just a few seconds. Let's try to visualize another channel from the same layer. The network can generate wide diversity of patterns.

In [ ]:
print(layer)
print(channel)

In [ ]:
# One neuron in whole layer with one neuron in higher channel

feat_layer2 = T('mixed3a_3x3_pre_relu')[:,5,5,50]
render_lapnorm2(T('mixed4e_5x5_pre_relu')[:,5,5,50], t_obj2=feat_layer2, score='mean', octave_n=4)

In [ ]:
# One neuron in whole layer with one neuron in higher channel WITH some negation of diversity?

feat_layer2 = T('mixed3a_3x3_pre_relu')[:,5,5,50] - 1e-1*tf.reduce_mean(T('mixed3a_3x3_pre_relu')[:,:,:,:], axis=3)
render_lapnorm2(T('mixed4e_5x5_pre_relu')[:,5,5,50] - 1e-1*tf.reduce_mean(T('mixed4e_5x5_pre_relu')[:,:,:,:], axis=3), t_obj2=feat_layer2, score='mean', octave_n=4)

In [ ]:
# One neuron in whole layer with higher channel WITH some negation of diversity? Trying to subtract entire layer with "hard" approach

feat_layer2 = 2*T('mixed3a_3x3_pre_relu')[:,5,5,50] - tf.reduce_mean(T('mixed3a_3x3_pre_relu')[:,:,:,:], axis=3)
render_lapnorm2(2*T('mixed4e_5x5_pre_relu')[:,5,5,50] - tf.reduce_mean(T('mixed4e_5x5_pre_relu')[:,:,:,:], axis=3), t_obj2=feat_layer2, score='mean', octave_n=8)

In [ ]:
#intra neuron and layer blending

feat_layer2 = T('mixed3a_3x3_pre_relu')[:,0:14,14:28,15:20]
render_lapnorm2(T('mixed4e_5x5_pre_relu')[:,0:7,6,50] + T('mixed4e_5x5_pre_relu')[:,7:14,13,30], t_obj2=feat_layer2, score='mean', octave_n=4)

In [ ]:
def get_mask_sizes(init_size, oct_n, oct_s):
    sizes = [ np.int32(np.float32(init_size)) ]
    for octave in range(oct_n-1):
        hw = np.float32(sizes[-1]) / oct_s
        sizes.append(np.int32(hw))
    sizes = list(reversed(sizes))
    return sizes

def lapnorm_multi(t_obj, mask, img0=img_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='clear', visfunc=visstd, ratio=None, morph_im=None, file_dir=os.getcwd()):
    mask_sizes = get_mask_sizes(mask.shape[0:2], oct_n, oct_s)
    img0 = resize(img0, np.int32(mask_sizes[0])) 
    t_score = [tf.reduce_mean(t) for t in t_obj] # defining the optimization objective
    t_grad = [tf.gradients(t, t_input)[0] for t in t_score] # behold the power of automatic differentiation!
    # build the laplacian normalization graph
    lap_norm_func = tffunc(np.float32)(partial(lap_normalize, scale_n=lap_n))
    img = img0.copy()
    for octave in range(oct_n):
        if octave>0:
            hw = mask_sizes[octave] #np.float32(img.shape[:2])*oct_s
            img = resize(img, np.int32(hw))
        oct_mask = resize(mask, np.int32(mask_sizes[octave]))
        for i in range(iter_n):
            g_tiled = [lap_norm_func(calc_grad_tiled(img, t, i)) for t in t_grad]
            for g, gt in enumerate(g_tiled):
                img += gt * step * oct_mask[:,:,g].reshape((oct_mask.shape[0],oct_mask.shape[1],1))
            print('.', end='')
        print("octave %d/%d"%(octave+1, oct_n))
        if vis == 'clear':
            clear_output()
            showarray(visfunc(img))
        if vis == 'all':
            showarray(visfunc(img))
        if vis == 'zoomin':
            crop = 25
            img_c = img
            # just for more color
            img_c += img*0.2
            img_c = img_c[crop:-crop, crop:-crop]
            print(img_c.shape)
            img_c = imresize(img_c, (len(img[0]), len(img[1]), 3), order=4,
                 clip=False, preserve_range=True)#[np.newaxis].astype(np.float32)
            img = img_c
            print(img.shape)
    if vis == 'fin':
        showarray(visfunc(img))
    if vis == 'save':
        img = visfunc(img)
        img = np.uint8(np.clip(img, 0, 1)*255)
        #io.imwrite(file_dir+'/interp%.3f.png' % (ratio), img)
        
        #for colab
        fname = '/interp%.3f.png' % (ratio)
        img_out = PIL.Image.fromarray(img)
        img_out.save(fname)
        files.download(fname)
        
    if vis == 'zoomin':
        img_c = visfunc(img_c)
        img_c = np.uint8(np.clip(img_c,0, 1)*255)
        #io.imwrite(file_dir+'/zinterp%.3f.png' % (wght1), img_c)
        
        #for colab
        fname = '/zinterp%.3f.png' % (ratio)
        img_out = PIL.Image.fromarray(img)
        img_out.save(fname)
        files.download(fname)
    if vis == 'morph':
        morph_im = imresize(morph_im, (img.shape), preserve_range=True)
        img = visfunc(img)
        img = np.uint8(np.clip(img, 0, 1)*255)
        imgs = np.asarray(morph(morph_im, img)).astype(np.uint8)
        morph_im = img
        for i, _ in enumerate(imgs):
            #io.imwrite(file_dir+'/interp%.3f.png' % (wght1), img)

            #for colab
            fname = '/minterp%.3f%.2d.png' % (ratio, i)
            img_out = PIL.Image.fromarray(imgs[i])
            img_out.save(fname)
            files.download(fname)
        return morph_im

In [ ]:
h, w = 400, 400
octave_n = 3
octave_scale = 1.4
iter_n = 10

!wget https://raw.githubusercontent.com/ml4a/ml4a-guides/master/assets/kitty.jpg

img0 = load_image('kitty.jpg', h, w)

objectives = [T('mixed4d_3x3_bottleneck_pre_relu')[:,:,:,25], 
              T('mixed5a_5x5_bottleneck_pre_relu')[:,:,:,29]]

# mask
mask = np.zeros((h, w, 2))
mask[:,:200,0] = 1.0
mask[:,200:,1] = 1.0

img1 = lapnorm_multi(objectives, img0, mask, iter_n, 1.0, octave_n, octave_scale)
display_image(img1)

In [ ]:
h, w = 512, 1024

img0 = np.random.uniform(size=(h, w, 3)) + 100.0

octave_n = 3
octave_scale = 1.4
objectives = [T('mixed3a_3x3_pre_relu')[:,:,:,22], 
              T('mixed4e_5x5_pre_relu')[:,:,:,50]]

mask = np.zeros((h, w, 2))
mask[:,:,0] = np.linspace(0,1,w)
mask[:,:,1] = np.linspace(1,0,w)


img1 = lapnorm_multi(objectives, mask, img0=img0, iter_n=25, step=1.0, oct_n=3, oct_s=1.4, lap_n=4)

In [ ]:
#frame by frame interp
mask = np.zeros((h, w, 2))
frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    mask[:,:,0] = i/frames
    mask[:,:,1] = (frames-i)/frames
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#fade extremes maintain sides
mask = np.zeros((h, w, 2))
frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    mask[:,:,0] =  np.linspace(0,1,w)*(i/frames)
    mask[:,:,1] = np.linspace(1,0,w)*((frames-i)/frames)
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#swap sides linearly
mask = np.zeros((h, w, 2))
frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    mask[:,:,0] =  np.linspace(i/frames,(frames-i)/frames,w)
    mask[:,:,1] = np.linspace((frames-i)/frames,i/frames,w)
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#sweep out sides linearly
mask = np.zeros((h, w, 2))
frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    mask[:,:,0] =  np.linspace(i/frames,1,w)
    mask[:,:,1] = np.linspace(0,(frames-i)/frames,w)
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#centered linear interp
mask = np.zeros((h, w, 2))
frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    mask[:,:,0] = np.linspace(i/frames,(frames-i)/frames,w)*np.linspace((frames-i)/frames,i/frames,w)
    mask[:,:,1] = np.linspace((frames-i)/frames,i/frames,w)*np.linspace(i/frames,(frames-i)/frames,w)
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#swap sides via center
mask = np.zeros((h, w, 2))

mu, std = 0.0, 0.25

frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc=mu, scale=std, size=(224,224,3))+100
    data = np.random.normal(loc=i/frames, scale=std, size=(w))
    mask[:,0,0] = np.linspace(i/frames,(frames-i)/frames,h)
    mask[0,:,0] = data
    mask[:,:,1] = np.linspace((frames-i)/frames,i/frames,w)
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
from scipy.stats import norm

fig, ax = plt.subplots(1, 1)

mu = 0.0
sigma = 0.25

x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), w)
data = norm.pdf(x, mu, sigma)
data /= np.max(data)

ax.plot(x, data,'r-', lw=5, alpha=0.6, label='norm pdf')

print(np.max(data))


In [ ]:
#swap sides via density/bell curve
mask = np.zeros((h, w, 2))
mu = 0.0
sigma = 0.5

x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), w)

frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc=mu, scale=std, size=(224,224,3))+100
    data = norm.pdf(x, (i/frames)*2-1, sigma)
    data /= np.max(data)
    #broadcasting right?
    mask[:,:,0] = np.ones((h,1))
    mask[:,:,0] = 1.0-data
    mask[:,:,1] = np.zeros((h,1))
    mask[:,:,1] = data
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
showarray(visstd(mask[:,:,0]*255))

In [ ]:
h, w = 500, 800

cy, cx = 0.5, 0.5

# circle masks
pts = np.array([[[i/(h-1.0),j/(w-1.0)] for j in range(w)] for i in range(h)])
ctr = np.array([[[cy, cx] for j in range(w)] for i in range(h)])
pts -= ctr
# change this for greater circle size
dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**0.5
dist = dist / np.max(dist)

# mask = np.ones((h, w, 2))
# for x in range(0, w):
#   for y in range(0, h):
#     mask[y, x, 0] = (4.0 * x / w) % 1.0
#     mask[y, x, 1] = (3.0 * y / h) % 1.0

mask = np.ones((h, w, 2))
mask[:, :, 0] = dist
mask[:, :, 1] = 1.0-dist


img0 = np.random.uniform(size=(h, w, 3)) + 100.0

octave_n = 3
octave_scale = 1.4
objectives = [T('mixed3a_3x3_pre_relu')[:,:,:,22], 
              T('mixed4e_5x5_pre_relu')[:,:,:,50]]


img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
frames = 10.0

mask = np.ones((h, w, 2))
mask[:, :, 0] = dist
mask[:, :, 1] = 1.0-dist

for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**(i/frames)
    dist = dist / np.max(dist)
    mask[:,:,0] = dist
    mask[:,:,1] = 1.0 - dist
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#hole in the middle

mask = np.ones((h, w, 2))
mask[:, :, 0] = dist
mask[:, :, 1] = 1.0-dist

frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**0.5
    mask[:,:,0] =  np.linspace(i/frames,(frames-i)/frames,w)*dist*0.25
    mask[:,:,1] = np.linspace((frames-i)/frames,i/frames,w)*dist*0.25
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#vortex side to side

frames = 10.0
for i in range(int(frames+1)):
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc =0.0, scale =0.25, size=(224,224,3))+100
    dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**0.5
    mask[:,:,0] =  np.linspace(i/frames,(frames-i)/frames,w)*dist*0.5
    mask[:,:,1] = np.linspace((frames-i)/frames,i/frames,w)*(1.0-dist)*0.5
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
#circular side to side

h, w = 500, 800

mask = np.ones((h, w, 2))

frames = 10.0
for i in range(int(frames+1)):
    cy, cx = 0.5, i/frames
    # circle masks
    pts = np.array([[[j/(h-1.0),k/(w-1.0)] for k in range(w)] for j in range(h)])
    ctr = np.array([[[cy, cx] for k in range(w)] for j in range(h)])
    pts -= ctr
    # change this for greater circle size
    dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**0.5
    dist = dist / np.max(dist)
    mask[:, :, 0] = dist
    mask[:, :, 1] = 1.0-dist
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc=mu, scale=std, size=(224,224,3))+100
    print(i/frames)
    img1 = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='fin')

In [ ]:
# setup read/write TO-DO: SETuUP DIFFERENT DIRECTORIES FOR EACH GIF

#file_dir = os.getcwd()+"/feat_interp"
#os.makedirs(file_dir, exist_ok=True)

# for colab
from google.colab import files

In [ ]:
#GIF WITH MORPH - circular side to side

h, w = 500, 800

mask = np.ones((h, w, 2))

# for gif jitter smoothing
morph_im = np.uint8(np.clip(visstd(norm_noise), 0, 1)*255)

frames = 100.0
for i in range(int(frames+1)):
    cy, cx = 0.5, i/frames
    # circle masks
    pts = np.array([[[j/(h-1.0),k/(w-1.0)] for k in range(w)] for j in range(h)])
    ctr = np.array([[[cy, cx] for k in range(w)] for j in range(h)])
    pts -= ctr
    # change this for greater circle size
    dist = (pts[:,:,0]**2 + pts[:,:,1]**2)**0.5
    dist = dist / np.max(dist)
    mask[:, :, 0] = dist
    mask[:, :, 1] = 1.0-dist
    # control stable noise (or not) with seed
    np.random.seed(0)
    norm_noise = np.random.normal(loc=mu, scale=std, size=(224,224,3))+100
    print(i/frames)
    morph_im = lapnorm_multi(objectives, mask, img0=norm_noise, iter_n=5, step=1.0, oct_n=3, oct_s=1.4, lap_n=4, vis='morph', ratio=i/frames, morph_im=morph_im)


In [ ]:
# Make our GIF

images = []
for file_name in os.listdir(file_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(file_dir, file_name)
        images.append(io.imread(file_path))
        print(np.asarray(images).shape)
io.mimsave(file_dir+'/interprow100-10.gif', images, subrectangles=True)

#ipyd.Image(url='interp10.gif',
           #height=500, width=500)

In [ ]:
io.help('gif')